In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

## Drop duplicates and incomplete rows

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

## Examine the schema

In [ ]:
df.printSchema()

## Rename columns

In [ ]:
# df1 = df.withColumnRenamed("Employee ID", "employee_id") \
#         .withColumnRenamed("Email", "email") \
#         .withColumnRenamed("Gender", "gender") \
#         .withColumnRenamed("Hire Date", "hire_date") \
#         .withColumnRenamed("DOB", "dob") \
#         .withColumnRenamed("Encrypted Password", "password")
# df1.show(5)
# NOTE: The above code from Homework Lesson 22.3.4 was not needed since all of the columns aligned to what was needed:)
df1=df
df1

## Create a new DataFrame for products 

In [ ]:
products = df1.select(["product_id", "product_title"])
products.show(5)

## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://rdsdb.cgx5vst360bw.us-east-2.rds.amazonaws.com:5432/dbforbigdata"
print("Need to change password below as per BZ Gmail email on 6/8/22 at 10:50pm")
print("change it from xxxxxxxx to temporary one 1 per BZ Gmail email on 6/8/22 at 10:50pm")
config = {"user":"postgres",
          "password": "tempone1",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

## Create a new DataFrame for review_id_table

In [ ]:
review_id_table = df1.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)

## Write DataFrame to RDS

In [ ]:
# Write DataFrame to review_id_table
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

## Create a new DataFrame for customers:

In [ ]:
# Since customer_count isn't available in the Amazon supplied Datasets, first need to count customer occurences of customer_id, then rename columns, then 
customers = df1.select(["customer_id"])
customers.show(10)

customers = customers.groupBy("customer_id").count()
customers.show(10)

customers.orderBy("customer_id").select(["customer_id", "count"])
customers.show(10)

customers = customers.withColumnRenamed("count","customer_count")
customers.show(10)

customers.printSchema()

## Write DataFrame to RDS

In [ ]:
# Write DataFrame to table
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

## Create a new DataFrame for vine_table


In [ ]:
vine_table = df1.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

In [ ]:
vine_table.printSchema()

## Change 'star_rating', 'helpful_votes', and 'total_votes' 
## in vine_table to Integer type

In [ ]:
#from traitlets.traitlets import Integer
from pyspark.sql.types import IntegerType
## Convert star_rating from string to interger
vine_table = vine_table.withColumn("star_rating",vine_table["star_rating"].cast(IntegerType()))\
  .withColumn("helpful_votes",vine_table["helpful_votes"].cast(IntegerType()))\
  .withColumn("total_votes",vine_table["total_votes"].cast(IntegerType()))
vine_table.printSchema()

## Write DataFrame to RDS

In [ ]:
# Write DataFrame to table
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)